In [1]:
import pandas as pd
import requests
from time import sleep

In [2]:
url  ='http://dadosabertos.rio.rj.gov.br/apiTransporte/apresentacao/rest/index.cfm/obterTodasPosicoes'

In [3]:
def get_data():
    try:
        data = requests.get(url).json()
        df = pd.DataFrame.from_records(data['DATA'], columns=data['COLUMNS'])
        df['DATAHORA'].apply(lambda x: pd.datetime.strptime(x, "%m-%d-%Y %H:%M:%S"))
        return df
    except:
        return pd.DataFrame()

In [4]:
get_data()

""


In [5]:
try:
    all_data = pd.read_csv('all_data.csv')
except:
    all_data = pd.DataFrame()


In [6]:
all_data.tail()

,DATAHORA,ORDEM,LINHA,LATITUDE,LONGITUDE,VELOCIDADE
72332,2020-04-05 15:44:18,A71557,409.0,-22.96372,-43.21699,0.0
72333,2020-04-05 15:44:19,A71615,100.0,-22.98479,-43.19864,0.0
72334,2020-04-05 15:44:19,B82626,678.0,-22.88632,-43.25134,16.0
72335,2020-04-05 15:44:20,A71523,409.0,-22.94313,-43.17628,42.0
72336,2020-04-05 15:44:20,B71074,455.0,-22.90287,-43.18370,14.0


In [10]:
y=0
while True:
    all_data = all_data.append(get_data(), ignore_index=True)
    all_data = all_data.drop_duplicates(keep='first')
    all_data.to_csv('all_data.csv', index=False)
    print(all_data.shape)
    data = {'api_key':'QEZAG779CCSVAYZ5' 'field2': all_data.shape[0]}
    thing_url ='https://api.thingspeak.com/update' #?api_key=QEZAG779CCSVAYZ5&field1=73
    resuests.post(thing_url,data)
    y += 1
    if y%10 == 0:
        !git pull
        !git add .
        !git commit -m "atualizando dados"
        !git push
    sleep(180)


(72337, 6)
Already up to date.


The file will have its original line endings in your working directory


[master 68c9eac] atualizando dados
 3 files changed, 5168 insertions(+), 5043 deletions(-)
 rewrite coletor_de_dados.ipynb (81%)
 create mode 100644 documentacao_gps.pdf


<Response [200]>